# Calculator

In [102]:
def pprint(*args, **kwargs):
    formatted_args = []
    for arg in args:
        if isinstance(arg, int):
            formatted_args.append(f"{arg:,}")
        elif isinstance(arg, float):
            formatted_args.append(f"{arg:,.2f}")
        else:
            formatted_args.append(arg)
    print(*formatted_args, **kwargs)

def calculate_all(title):

    print("*"*30, title)
    TPL = { e: tps*ledger_time for e, tps in TPS.items() }
    tpl_total = sum(TPL.values())

    LE_reads_per_op = {}
    LE_writes_per_op = {}
    LE_size = {}
    event_size_per_op = {}
    # avg size of classic LE
    LE_size['classic'] = 140

    if big_settings:
        #   classic
        #      max number of LE reads per op (vnext: limited DEX)
        LE_reads_per_op['classic'] = 10 if v_next else 1000
        #      max number of LE writes per op
        LE_writes_per_op['classic'] = LE_reads_per_op['classic']
        #      models each "write" as a token transfer
        event_size_per_op['classic'] = LE_writes_per_op['classic']*token_transfer_event_size

        #   soroban
        #      max number of LE reads
        LE_reads_per_op['soroban'] = 40
        #      max number of LE writes
        LE_writes_per_op['soroban'] = 25
        #      write as much as possible, use max allowed per tx
        #       note that this also impacts the total of amount read per tx
        #       but as tx also need to load wasm blobs, in average is probably fine
        LE_size['soroban'] = (129*1024)/LE_writes_per_op['soroban']
        #      max event produced
        event_size_per_op['soroban'] = 16*1024

        #   speedex
        LE_reads_per_op['speedex'] = 4
        LE_writes_per_op['speedex'] = 2
        LE_size['speedex'] = 512
        event_size_per_op['speedex'] = LE_writes_per_op['speedex']*token_transfer_event_size

        # tps getledgerentries (watcher node)
        gle_tps = 5
    else:
        #   classic
        #      avg number of LE reads per op 
        LE_reads_per_op['classic'] = 5
        #      avg number of LE writes per op
        LE_writes_per_op['classic'] = 5
        #      models each "write" as a token transfer
        event_size_per_op['classic'] = LE_writes_per_op['classic']*token_transfer_event_size

        #   soroban
        #      avg number of LE reads
        LE_reads_per_op['soroban'] = 10
        #      avg number of LE writes
        LE_writes_per_op['soroban'] = 5
        # use a 10th of capacity
        LE_size['soroban'] = (129*1024)/LE_writes_per_op['soroban']/10
        #       event produced is 50% of max
        event_size_per_op['soroban'] = 16*1024/2

        #    speedex
        LE_reads_per_op['speedex'] = 4
        LE_writes_per_op['speedex'] = 2
        #      assumes simple transfers only
        LE_size['speedex'] = 140
        event_size_per_op['speedex'] = LE_writes_per_op['speedex']*token_transfer_event_size

        # tps getledgerentries (watcher node)
        gle_tps = 2

    ###########
    # bucket list modeling

    # number of iops and bytes needed to read a single ledger entry
    # need to
    #    1. walk from recent to oldest buckets (up to bucket 19)
    #        bloom filter helps short circuit lookups
    #    2. for each bucket, page size on larger buckets -> more than 1 read occurs

    # additional extra reads (overhead)
    bl_avg_extra_nb_reads = 1

    if large_bl:
        # more Soroban entries in max mode
        bl_avg_le_size = 1024
    else:
        bl_avg_le_size = 300


    bl_avg_extra_bytes_read = bl_avg_le_size*bl_avg_extra_nb_reads

    bl_avg_nb_reads = 1+bl_avg_extra_nb_reads

    def bl_bytes_read_per_le(le_size):
        return le_size + bl_avg_extra_bytes_read

    ############
    # bucket list activity

    # size of an account LE
    le_account_bytes = 127

    ledger_LE_reads = {}
    ledger_bytes_read = {}
    ledger_LE_writes = {}
    ledger_bytes_write = {}

    # fees/seqnum
    ledger_LE_reads['fee'] = tpl_total*bl_avg_nb_reads
    ledger_bytes_read['fee'] = tpl_total*bl_bytes_read_per_le(le_account_bytes)
    ledger_LE_writes['fee'] = tpl_total
    ledger_bytes_write['fee'] = ledger_LE_writes['fee'] * le_account_bytes

    # per phase

    #   classic
    ledger_LE_reads['classic'] = TPL['classic']*LE_reads_per_op['classic']
    ledger_bytes_read['classic'] = ledger_LE_reads['classic']*bl_bytes_read_per_le(LE_size['classic'])
    ledger_LE_reads['classic'] *= bl_avg_nb_reads
    ledger_LE_writes['classic'] = TPL['classic']*LE_writes_per_op['classic']
    ledger_bytes_write['classic'] = ledger_LE_writes['classic']*LE_size['classic']

    #    soroban
    ledger_LE_reads['soroban'] = TPL['soroban']*LE_reads_per_op['soroban']
    ledger_bytes_read['soroban'] = ledger_LE_reads['soroban']*bl_bytes_read_per_le(LE_size['soroban'])
    ledger_LE_reads['soroban'] *= bl_avg_nb_reads
    ledger_LE_writes['soroban'] = TPL['soroban']*LE_writes_per_op['soroban']
    ledger_bytes_write['soroban'] = ledger_LE_writes['soroban']*LE_size['soroban']

    #    speedex
    ledger_LE_reads['speedex'] = TPL['speedex']*LE_reads_per_op['speedex']
    ledger_bytes_read['speedex'] = ledger_LE_reads['speedex']*bl_bytes_read_per_le(LE_size['speedex'])
    ledger_LE_reads['speedex'] *= bl_avg_nb_reads
    ledger_LE_writes['speedex'] = TPL['speedex']*LE_writes_per_op['speedex']
    ledger_bytes_write['speedex'] = ledger_LE_writes['speedex']*LE_size['speedex']

    # totals
    print('ledger_LE_reads:', ledger_LE_reads)
    print('ledger_bytes_read:', ledger_bytes_read)
    print('ledger_LE_writes:', ledger_LE_writes)
    print('ledger_bytes_write:', ledger_bytes_write)

    ledger_total_LE_reads = sum(ledger_LE_reads.values())
    ledger_total_bytes_read = sum(ledger_bytes_read.values())
    ledger_total_LE_writes = sum(ledger_LE_writes.values()) 
    ledger_total_bytes_write = sum(ledger_bytes_write.values())

    pprint("ledger_total_LE_reads: ", ledger_total_LE_reads)
    pprint("ledger_total_bytes_read: ", ledger_total_bytes_read)

    pprint("ledger_total_LE_writes: ", ledger_total_LE_writes)
    pprint("ledger_total_bytes_write: ", ledger_total_bytes_write)



    ##############
    # meta

    ledger_meta_events = {}
    ledger_meta_other = {}

    # classic
    ledger_meta_events['classic'] = TPL['classic']*event_size_per_op['classic']
    #  ledger changes
    ledger_meta_other['classic'] = ledger_bytes_write['classic']*2

    # soroban
    ledger_meta_events['soroban'] = TPL['soroban']*event_size_per_op['soroban']
    #  ledger changes
    ledger_meta_other['soroban'] = ledger_bytes_write['soroban']*2

    # speedex
    ledger_meta_events['speedex'] = TPL['speedex']*event_size_per_op['speedex']
    #  ledger changes
    ledger_meta_other['speedex'] = ledger_bytes_write['speedex']*2

    ledger_meta_events = sum(ledger_meta_events.values())
    ledger_meta_other = sum(ledger_meta_other.values())
    ledger_meta = ledger_meta_events + ledger_meta_other

    pprint("ledger_meta_events: ", ledger_meta_events)
    pprint("ledger_meta_other:", ledger_meta_other)
    pprint("ledger_meta:", ledger_meta)


    pprint("meta_events (MB/s): ", ledger_meta_events/(1024*1024*ledger_time))
    pprint("meta_other (MB/s):", ledger_meta_other/(1024*1024*ledger_time))
    pprint("meta (MB/s):", ledger_meta/(1024*1024*ledger_time))

    # Overlay activity per ledger period
    ##   bucket list (overhead from validity checks from reading the account entry ie seqnum/fee)
    ledger_overlay_LE_reads = tpl_total*flooding_recv_factor
    ledger_overlay_LE_bytes = ledger_overlay_LE_reads*bl_bytes_read_per_le(le_account_bytes)
    ledger_overlay_LE_reads *= bl_avg_nb_reads

    ##   bandwdith
    if v_next:
        overlay_fan_out = 10
    else:
        overlay_fan_out = 20

    if large_tier1:
        tier1_size = 100
    else:
        tier1_size = 25

    # how long is allocated per ledger to flooding
    ledger_overlay_processing_time_ms = ledger_time*1000

    # tx size calculations
    tx_bytes = {}
    tx_footprint = {}
    tx_no_footprint_size_bytes = {}

    ##  classic
    tx_bytes['classic'] = 200

    ## soroban
    tx_footprint['soroban'] = LE_reads_per_op['soroban']+LE_writes_per_op['soroban']

    ###    Soroban key size (as seen in footprints)
    LE_key_size_bytes_soroban = 80
    ###    Soroban tx size without footprint
    tx_no_footprint_size_bytes['soroban'] = 200

    tx_bytes['soroban'] = tx_no_footprint_size_bytes['soroban'] + (tx_footprint['soroban']*LE_key_size_bytes_soroban)

    ## speedex
    tx_footprint['speedex'] = LE_reads_per_op['speedex']+LE_writes_per_op['speedex']

    ###    speedex tx size without footprint
    tx_no_footprint_size_bytes['speedex'] = 200

    tx_bytes['speedex'] = tx_no_footprint_size_bytes['speedex'] + (tx_footprint['speedex']*LE_key_size_bytes_soroban)

    # SCP
    scp_message_bytes = 150
    # number of tx sets per ledger (should be 1, but due to timing issues more than 1 leader may nominate)
    ledger_scp_tx_set_count = 1.1

    ### tx flooding
    ##### use the same factor than recv
    ledger_tx_flooding = {e: tpl*flooding_recv_factor for e, tpl in TPL.items()}
    ledger_tx_flood_bytes = sum(ledger_tx_flooding[e]*tx_bytes[e] for e in tx_bytes.keys())

    pprint("flooding bytes (per connection): ", ledger_tx_flood_bytes)
    ledger_tx_flood_bytes *= overlay_fan_out

    ### SCP flooding
    #### 6 messages per tier1 org
    ledger_scp_messages = tier1_size*6
    ledger_scp_flood_bytes = ledger_scp_messages*scp_message_bytes
    #### tx set overhead per ledger
    ledger_txset_bytes = sum(TPL[e]*tx_bytes[e] for e in tx_bytes)
    
    pprint("TxSet bytes: ", ledger_txset_bytes)
    ledger_txset_flood_bytes = ledger_txset_bytes*ledger_scp_tx_set_count
    ledger_scp_flood_bytes += ledger_txset_flood_bytes

    ledger_scp_flood_bytes *= overlay_fan_out

    # totals
    ledger_flood_bytes = ledger_tx_flood_bytes+ledger_scp_flood_bytes

    flood_bps = ledger_flood_bytes / ledger_overlay_processing_time_ms * 1000

    pprint("Network bps (GBit/s): ", flood_bps*8/1024/1024/1024)

    ###############
    # calculates additional activity caused by watcher nodes
    # its impact is on "overlay activity" as it's happening outside of "apply"
    
    if not is_validator:
        # watchers expose the "getledgerentries" endpoint
        gle_reads = ledger_time*2
        gle_read_bytes = gle_reads*bl_bytes_read_per_le(bl_avg_le_size)
        gle_reads *= bl_avg_nb_reads

        ledger_overlay_LE_reads += gle_reads
        ledger_overlay_LE_bytes += gle_read_bytes

    ################
    #  apply step
    # apply is equivalent to the sequence [loads, classic, Soroban, writes]
    apply_time_ms_classic = TPL['classic']*tx_exec_time_ms['classic']
    if v_next:
        # overlay can use the entire time (background apply)
        ledger_overlay_time_ms = ledger_time*1000
        # calculates time available to read, classic is not parallel but rest is
        apply_other_exec_time_ms = (TPL['soroban']*tx_exec_time_ms['soroban'] + TPL['speedex']*tx_exec_time_ms['speedex'])/nb_exec_lanes
        apply_exec_time_ms = apply_time_ms_classic+apply_other_exec_time_ms
    else:
        ledger_overlay_time_ms = ledger_time*1000 - apply_time_ms
        apply_other_exec_time_ms = TPL['soroban']*tx_exec_time_ms['soroban']+TPL['speedex']*tx_exec_time_ms['speedex']
        apply_exec_time_ms = apply_time_ms_classic+apply_other_exec_time_ms

    pprint("apply_exec_time_ms: ", apply_exec_time_ms)

    apply_read_time_ms = apply_time_ms - apply_write_time_ms - apply_exec_time_ms
    assert apply_read_time_ms > 0, "TPS too high (apply work exceeds target)"
    apply_read_time_ms += apply_optimistic_read_time_ms

    pprint("apply_read_time_ms: ", apply_read_time_ms)
    pprint("apply_write_time_ms: ", apply_write_time_ms)
    # calculates max iops for overlay and apply separately

    iops_overlay = ledger_overlay_LE_reads*1000/ledger_overlay_time_ms
    iops_apply = ledger_total_LE_reads*1000/apply_read_time_ms

    rbps_overlay = ledger_overlay_LE_bytes*1000/ledger_overlay_time_ms

    rbps_apply = ledger_total_bytes_read*1000/apply_read_time_ms
    wbps_apply = ledger_total_bytes_write*1000/apply_write_time_ms

    pprint("rbps_apply (MB): ", rbps_apply/1024/1024)
    pprint("wbps_apply (MB): ", wbps_apply/1024/1024)

    if v_next:
        # parallel -> add
        max_iops = iops_overlay + iops_apply
        max_rbps = rbps_overlay + rbps_apply
        max_wbps = wbps_apply
        max_bps = max(rbps_overlay + rbps_apply, rbps_overlay + wbps_apply)
    else:
        # sequential -> max
        max_iops = max(iops_overlay, iops_apply)
        max_rbps = max(rbps_overlay, rbps_apply)
        max_wbps = wbps_apply
        max_bps = max(max_rbps, max_wbps)

    pprint("max iops: ", max_iops)
    pprint("max read Mbps: ", max_rbps/1024/1024)
    pprint("max write Mbps: ", max_wbps/1024/1024)
    pprint("max disk bandwidth Mbps: ", max_bps/1024/1024)

    # source: https://aws.amazon.com/ec2/instance-types/i3en
    max_4k_IOPS_NVMe = 2000000
    max_write_NVMe = 16*1024*1024*1024
    max_bps_NVMe=4096*max_4k_IOPS_NVMe
    assert max_iops < max_4k_IOPS_NVMe, "disk IOPS exceeded"
    assert max_bps < max_bps_NVMe, "disk bandwidth exceeded"
    assert max_wbps < max_write_NVMe, "disk linear write bandwidth exceeded"


# Main Settings

In [103]:
# set to True when using future work
# v_next = False should represent what core can do today
v_next = True

# set to True when modeling validators (uses archives)
# watchers expose endpoints for RPC that need to be modeled
is_validator = False

# worst case limits
big_settings = True

# True: BL contains large ledger entries
large_bl = True

# True: larger "tier1"
large_tier1 = True

# how long each tx type takes
tx_exec_time_ms = {'classic': 0.1, 'soroban': 8, 'speedex': 1}

# total round time
ledger_time = 5.0

# how much time is allocated to apply phase
apply_time_ms = 3000

# how much time can be spent reading without blocking apply
apply_optimistic_read_time_ms = 0

if v_next:
    nb_exec_lanes = 10
else:
    # do not define it as current does not support lanes
    1

# how long is allocated for writing
# (may want to make this a function of tps&footprints instead)
if v_next:
    apply_write_time_ms = 250
else:
    apply_write_time_ms = 100

# tx flooding multiplier (how many unique transactions get received in steady state)
flooding_recv_factor = 2.5

# other settings
# apply time (in addition to source account modified for seqnum & fees)

# how many bytes in a token transfer size
# "transfer", source, destination, asset issuer/contract, amount
token_transfer_event_size = 16+3*32+4*8+16


# Current network

In [105]:
TPS = {'classic': 200, 'soroban': 10, 'speedex': 0}
calculate_all("current")


****************************** current
ledger_LE_reads: {'fee': 2100.0, 'classic': 20000.0, 'soroban': 4000.0, 'speedex': 0.0}
ledger_bytes_read: {'fee': 1208550.0, 'classic': 11640000.0, 'soroban': 12615680.0, 'speedex': 0.0}
ledger_LE_writes: {'fee': 1050.0, 'classic': 10000.0, 'soroban': 1250.0, 'speedex': 0.0}
ledger_bytes_write: {'fee': 133350.0, 'classic': 1400000.0, 'soroban': 6604800.0, 'speedex': 0.0}
ledger_total_LE_reads:  26,100.00
ledger_total_bytes_read:  25,464,230.00
ledger_total_LE_writes:  12,300.00
ledger_total_bytes_write:  8,138,150.00
ledger_meta_events:  2,419,200.00
ledger_meta_other: 16,009,600.00
ledger_meta: 18,428,800.00
meta_events (MB/s):  0.46
meta_other (MB/s): 3.05
meta (MB/s): 3.52
flooding bytes (per connection):  1,175,000.00
TxSet bytes:  470,000.00
Network bps (GBit/s):  0.03
apply_exec_time_ms:  140.00
apply_read_time_ms:  2,610.00
apply_write_time_ms:  250
rbps_apply (MB):  9.30
wbps_apply (MB):  31.04
max iops:  11,054.00
max read Mbps:  9.88
ma

# Parallel Soroban

In [106]:
nb_exec_lanes = 15
TPS = {'classic': 200, 'soroban': 500, 'speedex': 0}
calculate_all("phase 1")

****************************** phase 1
ledger_LE_reads: {'fee': 7000.0, 'classic': 20000.0, 'soroban': 200000.0, 'speedex': 0.0}
ledger_bytes_read: {'fee': 4028500.0, 'classic': 11640000.0, 'soroban': 630784000.0, 'speedex': 0.0}
ledger_LE_writes: {'fee': 3500.0, 'classic': 10000.0, 'soroban': 62500.0, 'speedex': 0.0}
ledger_bytes_write: {'fee': 444500.0, 'classic': 1400000.0, 'soroban': 330240000.0, 'speedex': 0.0}
ledger_total_LE_reads:  227,000.00
ledger_total_bytes_read:  646,452,500.00
ledger_total_LE_writes:  76,000.00
ledger_total_bytes_write:  332,084,500.00
ledger_meta_events:  42,560,000.00
ledger_meta_other: 663,280,000.00
ledger_meta: 705,840,000.00
meta_events (MB/s):  8.12
meta_other (MB/s): 126.51
meta (MB/s): 134.63
flooding bytes (per connection):  34,250,000.00
TxSet bytes:  13,700,000.00
Network bps (GBit/s):  0.74
apply_exec_time_ms:  1,433.33
apply_read_time_ms:  1,316.67
apply_write_time_ms:  250
rbps_apply (MB):  468.23
wbps_apply (MB):  1,266.80
max iops:  175,9

# Faster consensus

In [107]:
ledger_time = 2.0
apply_time_ms = 1000
calculate_all("phase 2")

****************************** phase 2
ledger_LE_reads: {'fee': 2800.0, 'classic': 8000.0, 'soroban': 80000.0, 'speedex': 0.0}
ledger_bytes_read: {'fee': 1611400.0, 'classic': 4656000.0, 'soroban': 252313600.0, 'speedex': 0.0}
ledger_LE_writes: {'fee': 1400.0, 'classic': 4000.0, 'soroban': 25000.0, 'speedex': 0.0}
ledger_bytes_write: {'fee': 177800.0, 'classic': 560000.0, 'soroban': 132096000.0, 'speedex': 0.0}
ledger_total_LE_reads:  90,800.00
ledger_total_bytes_read:  258,581,000.00
ledger_total_LE_writes:  30,400.00
ledger_total_bytes_write:  132,833,800.00
ledger_meta_events:  17,024,000.00
ledger_meta_other: 265,312,000.00
ledger_meta: 282,336,000.00
meta_events (MB/s):  8.12
meta_other (MB/s): 126.51
meta (MB/s): 134.63
flooding bytes (per connection):  13,700,000.00
TxSet bytes:  5,480,000.00
Network bps (GBit/s):  0.74
apply_exec_time_ms:  573.33
apply_read_time_ms:  176.67
apply_write_time_ms:  250
rbps_apply (MB):  1,395.86
wbps_apply (MB):  506.72
max iops:  517,466.26
max r

# Speedex++

In [108]:
nb_exec_lanes = 15
TPS = {'classic': 200, 'soroban': 500, 'speedex': 500}
calculate_all("phase 3")

****************************** phase 3
ledger_LE_reads: {'fee': 4800.0, 'classic': 8000.0, 'soroban': 80000.0, 'speedex': 8000.0}
ledger_bytes_read: {'fee': 2762400.0, 'classic': 4656000.0, 'soroban': 252313600.0, 'speedex': 6144000.0}
ledger_LE_writes: {'fee': 2400.0, 'classic': 4000.0, 'soroban': 25000.0, 'speedex': 2000.0}
ledger_bytes_write: {'fee': 304800.0, 'classic': 560000.0, 'soroban': 132096000.0, 'speedex': 1024000.0}
ledger_total_LE_reads:  100,800.00
ledger_total_bytes_read:  265,876,000.00
ledger_total_LE_writes:  33,400.00
ledger_total_bytes_write:  133,984,800.00
ledger_meta_events:  17,344,000.00
ledger_meta_other: 267,360,000.00
ledger_meta: 284,704,000.00
meta_events (MB/s):  8.27
meta_other (MB/s): 127.49
meta (MB/s): 135.76
flooding bytes (per connection):  15,400,000.00
TxSet bytes:  6,160,000.00
Network bps (GBit/s):  0.83
apply_exec_time_ms:  640.00
apply_read_time_ms:  110.00
apply_write_time_ms:  250
rbps_apply (MB):  2,305.08
wbps_apply (MB):  511.11
max iops

# Bigger SKU

In [109]:
# c5ad.16xlarge equivalent: 64 vCPU (50+14 workers for other work)
# NVMe
# 20 GBit network
# https://aws.amazon.com/ec2/instance-types/c5/
nb_exec_lanes = 50

# optimistic reads
apply_optimistic_read_time_ms = 250

TPS = {'classic': 200, 'soroban': 1000, 'speedex': 9000}
calculate_all("phase 4")

****************************** phase 4
ledger_LE_reads: {'fee': 40800.0, 'classic': 8000.0, 'soroban': 160000.0, 'speedex': 144000.0}
ledger_bytes_read: {'fee': 23480400.0, 'classic': 4656000.0, 'soroban': 504627200.0, 'speedex': 110592000.0}
ledger_LE_writes: {'fee': 20400.0, 'classic': 4000.0, 'soroban': 50000.0, 'speedex': 36000.0}
ledger_bytes_write: {'fee': 2590800.0, 'classic': 560000.0, 'soroban': 264192000.0, 'speedex': 18432000.0}
ledger_total_LE_reads:  352,800.00
ledger_total_bytes_read:  643,355,600.00
ledger_total_LE_writes:  110,400.00
ledger_total_bytes_write:  285,774,800.00
ledger_meta_events:  39,168,000.00
ledger_meta_other: 566,368,000.00
ledger_meta: 605,536,000.00
meta_events (MB/s):  18.68
meta_other (MB/s): 270.07
meta (MB/s): 288.74
flooding bytes (per connection):  57,800,000.00
TxSet bytes:  23,120,000.00
Network bps (GBit/s):  3.10
apply_exec_time_ms:  720.00
apply_read_time_ms:  280.00
apply_write_time_ms:  250
rbps_apply (MB):  2,191.26
wbps_apply (MB):  1